In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# To use this experimental feature, we need to explicitly ask for it:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [4]:
# show categorical columns

for i in data:
    if data[i].nunique() < 7:
        
        print(i,data[i].nunique(),data[i].unique())

TARGET 2 [1 0]
NAME_CONTRACT_TYPE 2 ['Cash loans' 'Revolving loans']
CODE_GENDER 3 ['M' 'F' 'XNA']
FLAG_OWN_CAR 2 ['N' 'Y']
FLAG_OWN_REALTY 2 ['Y' 'N']
NAME_EDUCATION_TYPE 5 ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS 6 ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated'
 'Unknown']
NAME_HOUSING_TYPE 6 ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
FLAG_MOBIL 2 [1 0]
FLAG_EMP_PHONE 2 [1 0]
FLAG_WORK_PHONE 2 [0 1]
FLAG_CONT_MOBILE 2 [1 0]
FLAG_PHONE 2 [1 0]
FLAG_EMAIL 2 [0 1]
REGION_RATING_CLIENT 3 [2 1 3]
REGION_RATING_CLIENT_W_CITY 3 [2 1 3]
REG_REGION_NOT_LIVE_REGION 2 [0 1]
REG_REGION_NOT_WORK_REGION 2 [0 1]
LIVE_REGION_NOT_WORK_REGION 2 [0 1]
REG_CITY_NOT_LIVE_CITY 2 [0 1]
REG_CITY_NOT_WORK_CITY 2 [0 1]
LIVE_CITY_NOT_WORK_CITY 2 [0 1]
FONDKAPREMONT_MODE 4 ['reg oper account' nan 'org spec account' 'reg oper spec

In [5]:
# select some columns

data1=data[['TARGET','FLAG_OWN_CAR','NAME_EDUCATION_TYPE','NAME_CONTRACT_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','AMT_REQ_CREDIT_BUREAU_HOUR']]

data1.isnull().sum()

data1=data1.head(50000)

In [6]:
data1.head(3)

,TARGET,FLAG_OWN_CAR,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,AMT_REQ_CREDIT_BUREAU_HOUR
0,1,N,Secondary / secondary special,Cash loans,Single / not married,House / apartment,0.0
1,0,N,Higher education,Cash loans,Married,House / apartment,0.0
2,0,Y,Secondary / secondary special,Revolving loans,Single / not married,House / apartment,0.0


In [7]:
data1['FLAG_OWN_CAR'] = data1['FLAG_OWN_CAR'].map( {'Y': 1, 'N': 0} )

In [8]:
le = LabelEncoder()

for i in data1.columns:

    if data1[i].dtype in [object]:

        data1[i] = le.fit_transform(data1[i])
data1.head()

,TARGET,FLAG_OWN_CAR,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,AMT_REQ_CREDIT_BUREAU_HOUR
0,1,0,4,0,3,1,0.0
1,0,0,1,0,1,1,0.0
2,0,1,4,1,3,1,0.0
3,0,0,4,0,0,1,NaN
4,0,0,4,0,3,1,0.0


In [9]:
# imputing missing values with KNN imputer

knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(data1), columns=data1.columns)

In [35]:
X=df_knn_imputed[['FLAG_OWN_CAR','NAME_EDUCATION_TYPE','NAME_CONTRACT_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','AMT_REQ_CREDIT_BUREAU_HOUR']]

y=df_knn_imputed['TARGET']

In [36]:
# Split dataset into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [37]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9222666666666667


In [38]:
clf.predict([[ 0, 4, 0, 3, 1, 0]])

array([0.])

In [39]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test, y_pred)

array([[13834,     0],
       [ 1166,     0]], dtype=int64)

In [42]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(13834, 0, 1166, 0)